In [1]:
import numpy as np 
import pandas as pd 
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import requests
from datetime import datetime

C:\Users\kfuan\Anaconda3\envs\dl_fastai\lib\site-packages\tqdm\std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
stocks_list = pd.read_csv("../data/sp_500_stocks.csv")
stocks_list.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


In [4]:
momentum_portfolio = stocks_list.copy()

In [5]:
def chunks(input_list, n):
    for i in range(0, len(input_list), n):
        yield input_list[i: i+n]

In [6]:
batch_symbol = [name for name in chunks(stocks_list['Ticker'].values, 100)]

batch_strings = []
for batch in batch_symbol:
    batch_strings.append(",".join(batch))

In [7]:
batch_strings[0]

'A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA'

In [8]:
def batch_api_call(symbols):
    batch_api_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbols}&types=quote,stats&filter=latestPrice,month1ChangePercent,month3ChangePercent,month6ChangePercent,year1ChangePercent&token={IEX_CLOUD_API_TOKEN}"
    return requests.get(batch_api_url).json()

In [9]:
batch_api_call(batch_strings[0])

{'A': {'stats': {'month1ChangePercent': 0.04598142940919358,
   'month3ChangePercent': 0.1480415132291393,
   'month6ChangePercent': 0.2822058242350279,
   'year1ChangePercent': 0.5006526087528278},
  'quote': {'latestPrice': 124.68}},
 'AAL': {'stats': {'month1ChangePercent': 0.15570930053064067,
   'month3ChangePercent': 0.5987755534988347,
   'month6ChangePercent': 0.5231457916891553,
   'year1ChangePercent': -0.394661027571358},
  'quote': {'latestPrice': 17.5}},
 'AAP': {'stats': {'month1ChangePercent': 0.007557244095011529,
   'month3ChangePercent': 0.02105118135874485,
   'month6ChangePercent': 0.04175178867300727,
   'year1ChangePercent': 0.2174235351935744},
  'quote': {'latestPrice': 160.99}},
 'AAPL': {'stats': {'month1ChangePercent': 0.06235190693836823,
   'month3ChangePercent': 0.1986414785811856,
   'month6ChangePercent': 0.2696770357490322,
   'year1ChangePercent': 0.7713893906437327},
  'quote': {'latestPrice': 137.587}},
 'ABBV': {'stats': {'month1ChangePercent': 0.04

In [10]:
momentum_portfolio['Price'] = None

In [11]:
names_periods = ["month1", "month3", "month6", "year1"]
names_types = ["ChangePercent", "Percentiles"]

In [12]:
for i in names_periods:
    for j in names_types:
        momentum_portfolio[i + j] = None

In [13]:
momentum_portfolio

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles
0,A,None,None,None,None,None,None,None,None,None
1,AAL,None,None,None,None,None,None,None,None,None
2,AAP,None,None,None,None,None,None,None,None,None
3,AAPL,None,None,None,None,None,None,None,None,None
4,ABBV,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
500,YUM,None,None,None,None,None,None,None,None,None
501,ZBH,None,None,None,None,None,None,None,None,None
502,ZBRA,None,None,None,None,None,None,None,None,None
503,ZION,None,None,None,None,None,None,None,None,None


In [14]:
start = 0
for batch in tqdm_notebook(batch_strings):
    data = batch_api_call(batch)
    momentum_portfolio["Price"][start: start+len(data)] = momentum_portfolio['Ticker'][start:start+len(data)].apply(lambda x: data[x]['quote']['latestPrice'])
    
    for period in names_periods:
        name_ = period + names_types[0]
        momentum_portfolio[name_][start:start+len(data)] = momentum_portfolio['Ticker'][start:start+len(data)].apply(lambda x: data[x]['stats'][name_])
        
    start += len(data)

In [15]:
momentum_portfolio

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles
0,A,129.11,0.0478273,None,0.150623,None,0.284746,None,0.51337,None
1,AAL,17.8,0.154454,None,0.590688,None,0.526259,None,-0.405724,None
2,AAP,164.53,0.00744765,None,0.0206723,None,0.0424501,None,0.222231,None
3,AAPL,139.646,0.0634413,None,0.205319,None,0.258171,None,0.761995,None
4,ABBV,108.85,0.0417292,None,0.169908,None,0.180423,None,0.35994,None
...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.86,-0.0278882,None,0.0495034,None,0.134311,None,-0.01659,None
501,ZBH,164.44,0.0510658,None,0.18542,None,0.239527,None,0.0337611,None
502,ZBRA,408.14,0.078083,None,0.243252,None,0.451533,None,0.672472,None
503,ZION,48.59,0.126446,None,0.59317,None,0.549448,None,0.0755191,None


In [16]:
for period in names_periods:
    # adding percentiles columns
    momentum_portfolio[period+names_types[1]] = momentum_portfolio[period+names_types[0]].rank(pct=True)

In [17]:
momentum_portfolio

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles
0,A,129.11,0.0478273,0.526946,0.150623,0.518962,0.284746,0.626747,0.51337,0.898204
1,AAL,17.8,0.154454,0.912176,0.590688,0.952096,0.526259,0.872255,-0.405724,0.015968
2,AAP,164.53,0.00744765,0.327345,0.0206723,0.227545,0.0424501,0.215569,0.222231,0.660679
3,AAPL,139.646,0.0634413,0.628743,0.205319,0.616766,0.258171,0.590818,0.761995,0.972056
4,ABBV,108.85,0.0417292,0.499002,0.169908,0.542914,0.180423,0.465070,0.35994,0.808383
...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.86,-0.0278882,0.145709,0.0495034,0.285429,0.134311,0.369261,-0.01659,0.299401
501,ZBH,164.44,0.0510658,0.550898,0.18542,0.578842,0.239527,0.564870,0.0337611,0.375250
502,ZBRA,408.14,0.078083,0.682635,0.243252,0.662675,0.451533,0.816367,0.672472,0.946108
503,ZION,48.59,0.126446,0.842315,0.59317,0.956088,0.549448,0.886228,0.0755191,0.437126


In [18]:
names_percentiles = []
for period in names_periods:
    names_percentiles.append(period+names_types[1])

In [19]:
names_percentiles

['month1Percentiles',
 'month3Percentiles',
 'month6Percentiles',
 'year1Percentiles']

In [20]:
# HQM score is mean of all percentiles
momentum_portfolio['HQM score'] = momentum_portfolio[names_percentiles].mean(axis=1)

In [21]:
momentum_portfolio

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles,HQM score
0,A,129.11,0.0478273,0.526946,0.150623,0.518962,0.284746,0.626747,0.51337,0.898204,0.642715
1,AAL,17.8,0.154454,0.912176,0.590688,0.952096,0.526259,0.872255,-0.405724,0.015968,0.688124
2,AAP,164.53,0.00744765,0.327345,0.0206723,0.227545,0.0424501,0.215569,0.222231,0.660679,0.357784
3,AAPL,139.646,0.0634413,0.628743,0.205319,0.616766,0.258171,0.590818,0.761995,0.972056,0.702096
4,ABBV,108.85,0.0417292,0.499002,0.169908,0.542914,0.180423,0.465070,0.35994,0.808383,0.578842
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,106.86,-0.0278882,0.145709,0.0495034,0.285429,0.134311,0.369261,-0.01659,0.299401,0.274950
501,ZBH,164.44,0.0510658,0.550898,0.18542,0.578842,0.239527,0.564870,0.0337611,0.375250,0.517465
502,ZBRA,408.14,0.078083,0.682635,0.243252,0.662675,0.451533,0.816367,0.672472,0.946108,0.776946
503,ZION,48.59,0.126446,0.842315,0.59317,0.956088,0.549448,0.886228,0.0755191,0.437126,0.780439


In [22]:
#sort by HQM score
momentum_portfolio.sort_values(by='HQM score', ascending=False, inplace=True)
momentum_portfolio = momentum_portfolio[:50]
momentum_portfolio.reset_index(drop=True, inplace=True)

In [23]:
momentum_portfolio.head()

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles,HQM score
0,SIVB,496.05,0.275784,0.980040,0.679177,0.976048,1.18685,0.992016,0.962533,0.988024,0.984032
1,VIAC,53.01,0.42796,1.000000,0.792693,0.980040,1.02703,0.984032,0.61822,0.924152,0.972056
2,GM,55.59,0.339059,0.990020,0.537034,0.932136,1.14335,0.988024,0.610796,0.920160,0.957585
3,DISCA,43.03,0.355933,0.998004,1.03183,0.992016,0.898003,0.972056,0.396866,0.836327,0.949601
4,LB,50.43,0.216108,0.974052,0.435262,0.856287,0.955622,0.978044,1.06656,0.990020,0.949601


In [24]:
def equal_weight_position_size(n):
    global portfolio_size
    while True:
        try:
            portfolio_size = float(input("Enter the size of your portfolio: "))
            
            break
        except:
            print('Please input a correct number format.')
            continue
    global position_size
    position_size = portfolio_size/n

In [25]:
equal_weight_position_size(momentum_portfolio.shape[0])

Enter the size of your portfolio: 1000000


In [26]:
print(f"size of total portfolio: {portfolio_size} \nsize of each position: {position_size}")

size of total portfolio: 1000000.0 
size of each position: 20000.0


In [27]:
momentum_portfolio['#shares holding'] = np.floor(position_size/momentum_portfolio['Price'])

<ipython-input-27-700dab605eb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  momentum_portfolio['#shares holding'] = np.floor(position_size/momentum_portfolio['Price'])


In [28]:
momentum_portfolio.head(10)

,Ticker,Price,month1ChangePercent,month1Percentiles,month3ChangePercent,month3Percentiles,month6ChangePercent,month6Percentiles,year1ChangePercent,year1Percentiles,HQM score,#shares holding
0,SIVB,496.05,0.275784,0.980040,0.679177,0.976048,1.18685,0.992016,0.962533,0.988024,0.984032,40
1,VIAC,53.01,0.42796,1.000000,0.792693,0.980040,1.02703,0.984032,0.61822,0.924152,0.972056,377
2,GM,55.59,0.339059,0.990020,0.537034,0.932136,1.14335,0.988024,0.610796,0.920160,0.957585,359
3,DISCA,43.03,0.355933,0.998004,1.03183,0.992016,0.898003,0.972056,0.396866,0.836327,0.949601,464
4,LB,50.43,0.216108,0.974052,0.435262,0.856287,0.955622,0.978044,1.06656,0.990020,0.949601,396
5,AMAT,106.2,0.190608,0.954092,0.623169,0.960080,0.622429,0.912176,0.669119,0.944112,0.942615,188
6,FCX,30.5,0.100056,0.760479,0.683878,0.978044,1.34958,0.996008,1.53106,0.998004,0.933134,655
7,IVZ,22.46,0.292237,0.986028,0.627555,0.966068,1.28979,0.994012,0.310906,0.774451,0.930140,890
8,MOS,28.8,0.16475,0.926148,0.648728,0.972056,0.799317,0.952096,0.343713,0.794411,0.911178,694
9,ALGN,622.27,0.168877,0.934132,0.283621,0.714571,1.15297,0.990020,1.46188,0.996008,0.908683,32


## Try saving excel to multiple sheets

In [29]:
writer = pd.ExcelWriter('HQM_score_momentum.xlsx', engine='xlsxwriter')

In [30]:
# adding sheet consecutively
momentum_portfolio.to_excel(writer, index=False, sheet_name ='HQM')
momentum_portfolio.to_excel(writer, index=False, sheet_name='HQM_copy')

In [31]:
writer.save()